In [1]:
from scrapy import Selector
import requests
from tqdm import tqdm

In [2]:
full_url = 'http://www.edu.xunta.gal/centros/iesisaacdiazpardo/'
base_url = 'http://www.edu.xunta.gal'

output_dir = 'output'

In [3]:
def output_file(path, content):
    import os
    import errno
    
    filename = output_dir + '/' + path

    if not os.path.exists(os.path.dirname(filename)):
        try:
            os.makedirs(os.path.dirname(filename))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

    with open(filename, "w") as f:
        f.write(content)


In [4]:
def get_html(url):
    return requests.get(full_url).text

In [5]:
def get_sources_links(html):
    
    links = []
    
    css_links = Selector(text=html).xpath('//link/@href').getall()
    js_links = Selector(text=html).xpath('//script/@src').getall()
    img_links = Selector(text=html).xpath('//img/@src').getall()

    links.extend(css_links)
    links.extend(js_links)
    links.extend(img_links)

    return links

In [6]:
def save_html(html):
    import json
    json_html = json.dumps({'html': html})
        
    output_file('./index.html', html)
    output_file('./index.html.json', json_html)

In [7]:
def parse_links(links): 
    parsed_links = []
    
    for link in links:
        if(link[:5] == 'data:'):
            continue
        
        if (link[:7] != 'http://'):
            parsed_links.append(base_url + link)
        
        else:
            parsed_links.append(link)

    return parsed_links

In [23]:
def save_sources(source_links):
    for link in tqdm(source_links):
        try:
            source = requests.get(link).text

            _, path = link.split(base_url)

            path = path.split('?')[0]

            output_file(f'./{path}', source)
            
        except Exception as e: 
            print(e)

In [25]:
html = get_html(base_url)
save_html(html)

unparsed_links = get_sources_links(html)
links = parse_links(unparsed_links)

save_sources(links)

 88%|████████▊ | 53/60 [00:04<00:00, 12.65it/s]

HTTPConnectionPool(host='www.edu.xunta.galhttps', port=80): Max retries exceeded with url: //www.edu.xunta.es/espazoAbalar/sites/espazoAbalar/files/images/abalar-mobil-es.png (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2388c422d0>: Failed to establish a new connection: [Errno -2] Name or service not known'))


 98%|█████████▊| 59/60 [00:05<00:00, 10.50it/s]

not enough values to unpack (expected 2, got 1)
HTTPSConnectionPool(host='www.edu.xunta.gal', port=443): Max retries exceeded with url: /biblioteca/blog/files/BE_web_transp_150.png (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1076)')))


100%|██████████| 60/60 [00:05<00:00, 10.52it/s]
